In [1]:
#import dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
import keras

In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\users\admin\anaconda3\lib\site-packages (2.2.0)
Note: you may need to restart the kernel to use updated packages.


In [3]:
#load the dataset
file=open("frankenstein-2.txt").read()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 77: character maps to <undefined>

In [4]:
file=open("frankenstein-2.txt",encoding="utf8").read()

In [5]:
#tokenization
#standardization
def tokenize_words(input):
    input=input.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(input)
    filtered=filter(lambda token:token not in stopwords.words('english'),tokens)#removal of stopwords
    return "".join(filtered)
processed_inputs=tokenize_words(file)
                             

In [6]:
#neural network works with numbers so we need to convert characters into numbers
#chars to numbers
chars=sorted(list(set(processed_inputs)))
char_to_num=dict((c,i) for i,c in enumerate(chars)) 

In [7]:
#check if words to chars or chars to num has worked
input_len=len(processed_inputs)
vocab_len=len(chars)
print("total no of characters:",input_len)
print("total vocab:",vocab_len)

total no of characters: 223375
total vocab: 41


In [8]:
#seq length
seq_length=100
x_data=[]
y_data=[]


In [9]:
#loop throgh the seq
for i in range(0,input_len-seq_length,1):
    
    in_seq=processed_inputs[i:i+seq_length]
    out_seq=processed_inputs[i+seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns=len(x_data)
print("total patterns:",n_patterns)
    
    
    

total patterns: 223275


In [10]:
#convert input seq to numpy array
import numpy
X=numpy.reshape(x_data,(n_patterns,seq_length,1))
X=X/float(vocab_len)

In [11]:
#one hot encoding
y=np_utils.to_categorical(y_data)

In [12]:
#creating a seq model
model=Sequential()
model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))


In [13]:
#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer= 'adam')


In [14]:
#saving weights
filepath = 'model_weights_saved.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only= True, mode='min')
desired_callbacks =[checkpoint]

In [15]:
#fit model and let it train
model.fit(X,y,epochs=1, batch_size=256, callbacks = desired_callbacks)

Epoch 1/1
223275/223275 [==============================] - 5522s 25ms/step - loss: 2.9545

Epoch 00001: loss improved from inf to 2.95453, saving model to model_weights_saved.hdf5


In [16]:
#recompile the model with the saved weights
filename= "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss="categorical_crossentropy", optimizer='adam')


In [17]:
#output of the model back into characters
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [18]:
#random seed to help generate
start = numpy.random.randint(0, len(x_data)-1)
pattern = x_data[start]
print("Random Seed: ")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed: 
" tionfuriespossessedconcludedragechokedutteranceansweredstillnessnightloudfiendishlaughrangearslonghe "


In [19]:
# generate the text
for i in range(1000):
  x = numpy.reshape(pattern,(1,len(pattern), 1))
  x = x/float(vocab_len)
  prediction = model.predict(x,verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee